# Rotation of an image

This server implements the rotation of an image with zmq and protobuf.

The protobuf message is very simple and consists of an `image` message with png bytes as field.

In [1]:
!protoc --python_out=. image.proto

The server receives an images, parses it, decodes the png format.
After this, the image is rotated, encoded and serialized to send it back.

In [ ]:
import time
import zmq

import image_pb2
import numpy as np
import cv2

context = zmq.Context()

try:
    with context.socket(zmq.REP) as socket:
        socket.bind("tcp://*:5556")

        while True:
            #  Wait for next request from client
            message = socket.recv()
            print("Received image")

            img = image_pb2.image()
            img.ParseFromString(message)

            phot90 = cv2.imdecode(np.frombuffer(img.png,dtype=np.byte),cv2.IMREAD_COLOR)
            phot90 = cv2.rotate(phot90, cv2.ROTATE_90_CLOCKWISE)

            r, buf = cv2.imencode(".png", phot90)
            img.png = buf.tobytes()

            #  Send reply back to client
            socket.send(img.SerializeToString())
finally:
    context.term()

Received image
Received image
Received image
Received image
Received image


You can interrupt the server if you're done